<a href="https://colab.research.google.com/github/vkorennoy/AI_Dungeon/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22RuGPT3FinetuneHF_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Nov 13 10:01:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install torch -U # ==1.4.0
!pip3 install transformers -U # ==3.5.0

     |████████████████████████████████| 3.1 MB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 51.9 MB/s 
     |████████████████████████████████| 59 kB 8.5 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.0
    Uninstalling transformers-3.5.0:
      Successfully uninstalled transformers-3.5.0


In [3]:
import random
import requests
from time import sleep
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import logging


tokenizer = GPT2Tokenizer.from_pretrained("ekkasilina/big_baseline")
model = GPT2LMHeadModel.from_pretrained("ekkasilina/big_baseline")


def gen_fragment(context, bad_word_ids=[[1]], print_debug_output=False):
    input_ids = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt")
    input_ids = input_ids[:, -1700:]
    input_size = input_ids.size(1)
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=75 + input_size,
        min_length=50 + input_size,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        temperature=1.0,
        pad_token_id=0,
        eos_token_id=2,
        bad_words_ids=bad_word_ids,
        no_repeat_ngram_size=6
    )
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    generated_sequence = output_sequences[0].tolist()[input_size:]
    if print_debug_output:
        for idx in generated_sequence:
            print(idx, tokenizer.decode([idx], clean_up_tokenization_spaces=True).strip())
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    text = text[: text.find("</s>")]
    text = text[: text.rfind(".") + 1]
    return text



# ans = ''
# while True:
#     tmp = input().strip()

#     print("MY TEXT",tmp)

#     ans += tmp

#     new_story = gen_fragment(ans)
#     print("GEN TEXT: ", new_story)

#     ans += new_story

In [3]:
ans = ''
while True:
    tmp = input().strip()

    if tmp == "quit":
        break

    print("MY TEXT",tmp)

    ans += tmp

    new_story = gen_fragment(ans)
    print("GEN TEXT: ", new_story)

    ans += new_story

quit


In [3]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt2large")

model = AutoModel.from_pretrained("sberbank-ai/rugpt2large")

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at sberbank-ai/rugpt2large were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")

train_path = "drive/MyDrive/AI-dungeon/datasets/mix_translations_cyoa_fantasy_fairitailes.txt"

In [5]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=1024)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, data_collator

train_dataset, data_collator = load_dataset(train_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [6]:
from transformers import Trainer, TrainingArguments

In [ ]:
num_layers_to_unfreeze = 5
total_num_layers = 23

for param in model.parameters():
    param.requires_grad = False

for i in range(total_num_layers, total_num_layers - num_layers_to_unfreeze, -1):
    for param in model.transformer.h[i].parameters():
        param.requires_grad = True

In [7]:
training_args = TrainingArguments(
    output_dir="drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=4, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    # eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    save_total_limit = 3,
    )


trainer = Trainer(
    model=model.cuda(),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 14751
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 59004


Step,Training Loss
500,3.048800
1000,3.024600
1500,2.974300
2000,2.980800
2500,2.920800
3000,2.925700
3500,2.886300
4000,2.872400
4500,2.834100
5000,2.823300


Saving model checkpoint to drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-1000
Configuration saved in drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-2000
Configuration saved in drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-2000/config.json
Model weights saved in drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-12000] due to args.save_total_limit
Saving model checkpoint to drive/MyDrive/AI-dungeon/models/VK_test_mixed_dataset_gpt2_large/checkpoint-3000
Configuration saved in drive/MyDrive/AI-dungeon/models/VK_tes

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1)
# eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [7]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=3e-4)

In [8]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [9]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device);

In [171]:
outputs.shape

AttributeError: ignored

In [172]:
batch.size()

torch.Size([2, 64])

In [173]:
outputs.loss

In [10]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = batch.to(device)
        outputs = model(batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/354030 [00:00<?, ?it/s]

AttributeError: ignored